In [ ]:
#  FINAL CODE #

In [2]:
import pygame
import sys
import chess
from copy import deepcopy
import chess.polyglot


WIDTH = 800
WIN = pygame.display.set_mode((WIDTH, WIDTH))
pygame.display.set_caption("Chess")
board = chess.Board()


# Stuff for GUI

PEACH = (240, 217, 181)
BROWN = (189, 129, 79)
YELLOW = (255, 232, 128)
BLACK = (0, 0, 0)

#Load piece images
pieces = {'p': pygame.image.load('.\\GUI\\b_pawn.png'),
          'n': pygame.image.load('.\\GUI\\b_knight.png'),
          'b': pygame.image.load('.\\GUI\\b_bishop.png'),
          'r': pygame.image.load('.\\GUI\\b_rook.png'),
          'q': pygame.image.load('.\\GUI\\b_queen.png'),
          'k': pygame.image.load('.\\GUI\\b_king.png'),
          'P': pygame.image.load('.\\GUI\\w_pawn.png'),
          'N': pygame.image.load('.\\GUI\\w_knight.png'),
          'B': pygame.image.load('.\\GUI\\w_bishop.png'),
          'R': pygame.image.load('.\\GUI\\w_rook.png'),
          'Q': pygame.image.load('.\\GUI\\w_queen.png'),
          'K': pygame.image.load('.\\GUI\\w_king.png'),
          }

convert = {}
alphabets = ["a","b","c","d","e","f","g","h"]
for i in range(0,8):
    for j in range(0,8):
        convert.update({str(j)+str(i) : alphabets[j]+str(8-i)})

convert2 = {value:key for key, value in convert.items()}


def convert_sq_num(row,col):
    return chess.parse_square(convert[str(row) + str(col)])

class Node:
    def __init__(self, row, col, width):
        self.row = row
        self.col = col
        self.x = int(row * width)
        self.y = int(col * width)
        self.colour = PEACH

    def draw(self, WIN):
        pygame.draw.rect(WIN, self.colour, (self.x, self.y, WIDTH / 8, WIDTH / 8))
    def draw_piece(self,WIN,piece):
        WIN.blit(pieces[str(piece)],(self.x,self.y, WIDTH / 8, WIDTH / 8))

def make_grid(rows, width):
    grid = []
    gap = WIDTH // rows
    print(gap)
    for i in range(0, rows):
        grid.append([])
        for j in range(0, rows):
            node = Node(j, i, gap)
            grid[i].append(node)
            if (i+j)%2 ==1:
                grid[i][j].colour = BROWN
    return grid

def draw_grid(win, rows, width):
    gap = width // 8
    for i in range(rows):
        pygame.draw.line(win, BLACK, (0, i * gap), (width, i * gap))
        for j in range(rows):
            pygame.draw.line(win, BLACK, (j * gap, 0), (j * gap, width))  

def update_display(win, grid, rows, width):

    for row in grid:
        for spot in row:
            spot.draw(win)
            piece = board.piece_at(convert_sq_num(spot.row,spot.col))
            if piece == None:
                pass
            else:
                spot.draw_piece(WIN,piece)

def Find_Node(pos, WIDTH):
    interval = WIDTH / 8
    y, x = pos
    rows = y // interval
    columns = x // interval
    return int(rows), int(columns)

def display_potential_moves(positions, grid):
    """
    Displays all the potential moves
    """
    for i in positions:

        col = int(convert2[i.uci()[2]+i.uci()[3]][0])
        row = int(convert2[i.uci()[2]+i.uci()[3]][1])
        grid[row][col].colour = YELLOW

def remove_highlight(grid):
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            if (i+j)%2 == 0:
                grid[i][j].colour = PEACH
            else:
                grid[i][j].colour = BROWN
    return grid


# Stuff for Gameplay
# Initialising material scores

def evaluate(board):
    
    def count_mat_score(color):
        pawnValue = 100
        knightValue = 300
        bishopValue = 300
        rookValue = 500
        queenValue = 900
        
        material = 0
        material += len(board.pieces(chess.PAWN,color)) * pawnValue
        material += len(board.pieces(chess.KNIGHT,color)) * knightValue
        material += len(board.pieces(chess.BISHOP,color)) * bishopValue
        material += len(board.pieces(chess.ROOK,color)) * rookValue
        material += len(board.pieces(chess.QUEEN,color)) * queenValue

        return material

    white_mat_score = count_mat_score(chess.WHITE)
    black_mat_score = count_mat_score(chess.BLACK)
    
    #If positive, then better for white, if negative then better for black.
    eval_score = white_mat_score - black_mat_score
    
    return eval_score

def search_1(board):
    scores = []
    
    moves = list(board.legal_moves)
    
    for move in moves:
        temp_board = deepcopy(board)

        temp_board.push(move)
        scores.append(evaluate(temp_board))

        if board.turn == True:
            best_move = moves[scores.index(max(scores))]

        else:
            best_move = moves[scores.index(min(scores))]

    return best_move

def search_N(board,N):
    scores = []
    
    moves = list(board.legal_moves)
    
    for move in moves:
        temp_board = deepcopy(board)
        temp_board.push(move)
        
        #here we must check that the game is not over
        outcome = temp_board.outcome()
        
        #if checkmate
        if outcome == None:
            #if we have not got to the final depth
            #we search more moves ahead
            if N>1:
                temp_best_move = search_N(temp_board,N-1)
                temp_board.push(temp_best_move)

            scores.append(evaluate(temp_board))

        #if checkmate
        elif temp_board.is_checkmate():

            # we return this as best move as it is checkmate
            return move

        # if stalemate
        else:
            #value to disencourage a draw
            #the higher the less likely to draw
            #default value should be 0
            val = 1000
            if board.turn == True:
                scores.append(-val)
            else:
                scores.append(val)

        # This is the secondary eval function
        scores[-1] = scores[-1] + eval_space(temp_board)
        
    
    if board.turn == True:
       
        best_move = moves[scores.index(max(scores))]

    else:
        best_move = moves[scores.index(min(scores))]

    return best_move

def opening(board):
    
    reader = chess.polyglot.open_reader('baron30.bin')
    opening_move = reader.get(board)
    if opening_move == None:
        pass
    else:
        return opening_move.move
    return None
    
def eval_space(BOARD):
    no_moves = len(list(BOARD.legal_moves))

    #this function is always between 0 and 1 so we will never evaluate
    #this as being greater than a pawns value. The 20 value is arbitrary
    #but this number is chosen as it centers value around 0.5
    value = (no_moves/(20+no_moves))
    
    if BOARD.turn == True:
        return value
    else:
        return -value


running = True
selected = False
status = True
player_turn = 1
move_counter = 0

grid = make_grid(8,WIDTH)
update_display(WIN, grid, 8, WIDTH)
draw_grid(WIN,8,WIDTH)
pygame.display.update()

while running:
    pygame.time.delay(50)
    for event in pygame.event.get():

        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

        if status == True:
            if board.outcome() != None:
                print(board.outcome())
                pygame.display.set_caption("Game Over!")
                status = False
            
            if selected == False:
                remove_highlight(grid)
                update_display(WIN, grid, 8, WIDTH)
                draw_grid(WIN,8,WIDTH)
                pygame.display.update()
            
            # Player's Turn
            if player_turn == 1 and status == True:

                if event.type == pygame.MOUSEBUTTONDOWN:
                    pos = pygame.mouse.get_pos()
                    row, col = Find_Node(pos, WIDTH)
                    sq_num = convert_sq_num(row,col)

                    if selected == False:
                        remove_highlight(grid)
                        if board.piece_at(sq_num) != None:

                            legal_positions = []

                            for m in board.legal_moves:
                                if sq_num == m.from_square:
                                    legal_positions.append(m)

                            display_potential_moves(legal_positions,grid)
                            selected_sq = (row,col)
                            selected = True
                        update_display(WIN, grid, 8, WIDTH)
                        draw_grid(WIN,8,WIDTH)


                    elif selected == True:

                        sel_sq_num = convert_sq_num(selected_sq[0],selected_sq[1])

                        for m in legal_positions:
                            if (sel_sq_num == m.from_square and sq_num == m.to_square):
                                board.push(m)
                                update_display(WIN, grid, 8, WIDTH)
                                draw_grid(WIN,8,WIDTH)
                                selected = False
                                player_turn = 0
                                break
                        else:
                            selected = False
                            remove_highlight(grid)
                            update_display(WIN, grid, 8, WIDTH)
                            draw_grid(WIN,8,WIDTH)

                pygame.display.update()

            # Computer's Turn
            elif player_turn == 0 and status == True:
                best_move = opening(board)
                if best_move == None:
                    best_move = search_N(board,2)
                board.push(best_move)
                update_display(WIN, grid, 8, WIDTH)
                draw_grid(WIN,8,WIDTH)
                pygame.display.update()
                player_turn = 1

100


SystemExit: 